In [6]:
# 함수 로드
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [16]:
# 검색어와 제외 검색어 리스트
search_name_list = ["아이폰_12_64GB", "아이폰_12_128GB", "아이폰_12_256GB"]
search_except_name_list = ["미니, 프로", "미니, 프로", "미니, 프로"]
num = len(search_name_list)

In [17]:
for i in range(num):
    time.sleep(3)
    # webdriver 불러오기, '네이버 쇼핑' 접속
    driver = webdriver.Chrome('chromedriver')
    url = "https://shopping.naver.com/home/p/index.naver"
    driver.get(url)
    time.sleep(3)
    
    # 네이버 쇼핑에 search_name_list의 요소들 하나씩 검색
    # 상세 검색으로 필요없는 단어 설정해서 검색하기 과정
    search_name = search_name_list[i]
    search_except_name = search_except_name_list[i]
    
    # 상세 검색 여는 버튼 xpath = '//*[@id="_detailSearchButton"]'
    # 상세 검색의 기본검색 창 xpath = '//*[@id="lb_srh_basis"]'
    # 제외하는 단어 검색창 xpath= '//*[@id="lb_srh_exclude"]'
    # 상세 검색 버튼 xpath = '//*[@id="_detailSearchDiv"]/div/div[2]/a[3]'
    # 정확히 검색창 xpath = '//*[@id="lb_srh_exact"]'
    to_detail_search_button = driver.find_element_by_xpath('//*[@id="_detailSearchButton"]')
    detail_search_basic_search = driver.find_element_by_xpath('//*[@id="lb_srh_basis"]')
    except_word = driver.find_element_by_xpath('//*[@id="lb_srh_exclude"]')
    detail_search_button = driver.find_element_by_xpath('//*[@id="_detailSearchDiv"]/div/div[2]/a[3]')
    exact_search = driver.find_element_by_xpath('//*[@id="lb_srh_exact"]')
    
    # 상세 검색창에 검색어 입력
    to_detail_search_button.click()
    time.sleep(3)
    detail_search_basic_search.send_keys(search_name)
    exact_search.send_keys(search_name)
    
    # 상세 검색에 제외 단어 입력하기
    except_word.send_keys(search_except_name)
    detail_search_button.click()
    
    # 상품의 가격 수집(BS)
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    
    # 전체 상품 리스트 담기 태그 : 'ul', class_='list_basis'
    all_mer_list = soup.find('ul', class_='list_basis')
    
    # 가격표시 강조 부분 태그 : 'strong', class_='basicList_price__2r23_'
    basiclist_price = all_mer_list.find_all('strong', class_='basicList_price__2r23_')
    
    # 상품별 가격 모으기
    price_list = []
    for one in basiclist_price:
        tmp = one.find('span', class_='price_num__2WUXn').text
        price_list.append(tmp)
        
    # 선택된 상품의 평균, 최소, 최대값 구하기
    mod_price_list = []
    for i in list(price_list):
        tmp = i.replace("원", "")
        tmp = tmp.replace(",","")
        tmp = int(tmp)
        mod_price_list.append(tmp)
        
    # 평균, 최고, 최저가 출력
    mean = int(sum(mod_price_list) / len(mod_price_list))
    maxium = max(mod_price_list)
    minium = min(mod_price_list)
    print(f'{search_name}의 평균가격 :', mean,'원')
    print(f'{search_name}의 최고가격 :', maxium,'원')
    print(f'{search_name}의 최저가격 :', minium,'원')
    
    # 평균, 최고, 최저가 순서대로 리스트로 만들기
    mod_price = []
    mod_price.append(mean)
    mod_price.append(maxium)
    mod_price.append(minium)
    
    # 파일로 만들기
    import pandas as pd
    dict_dat = {search_name : mod_price}
    dat = pd.DataFrame(dict_dat)
    dat.to_csv(f"{search_name}_price.csv",index=False)
    dat.to_excel(f"{search_name}_price.xlsx",index=False)

아이폰_12_64GB의 평균가격 : 709894 원
아이폰_12_64GB의 최고가격 : 924990 원
아이폰_12_64GB의 최저가격 : 359000 원
아이폰_12_128GB의 평균가격 : 687726 원
아이폰_12_128GB의 최고가격 : 1142600 원
아이폰_12_128GB의 최저가격 : 305490 원
아이폰_12_256GB의 평균가격 : 974034 원
아이폰_12_256GB의 최고가격 : 1261000 원
아이폰_12_256GB의 최저가격 : 359000 원
